# QPAM |👷🏻‍♀️>: Generate a Q# project

In [1]:
from qbob import qbob, qpam
from qbob.intrinsics import Message, M, H

In [2]:
my_qbob = qbob.OperationBuilder("HelloWorld", entrypoint=True, debug=True)
my_qbob += Message("Hello World!")
with my_qbob.allocate_qubits("q", 1) as q:
    my_qbob += H(q)
    my_qbob.log_state(q)
    my_qbob.returns(M(q))

In [3]:
my_qpam = qpam.ProgramArchitect("MyProject")
my_qpam.add_operations(my_qbob)
project_file = my_qpam.create_project(".")

Preparing Q# environment...
Hello World!


In [4]:
!cat $project_file

<Project Sdk="Microsoft.Quantum.Sdk/0.14.2011120240">
    <PropertyGroup>
        <OutputType>Exe</OutputType>
        <TargetFramework>netcoreapp3.1</TargetFramework>
    </PropertyGroup>
</Project>

In [5]:
!cat Program.qs

namespace MyProject {
    open Microsoft.Quantum.Canon;
    open Microsoft.Quantum.Intrinsic;

    @EntryPoint()
    operation HelloWorld () : Result {
        Message("Hello World!");
        using (q = Qubit()) {
            H(q);
            // # wave function for qubits with ids (least to most significant): 0
            // ∣0❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
            // ∣1❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
            return M(q);
        }
    }
}

In [11]:
!dotnet run --project $project_file

========Hello World!
Zero
=

## With debugging

In [7]:
from qbob import qbob, qpam
from qbob.intrinsics import M, H, X

In [8]:
my_qbob = qbob.OperationBuilder("TestDumpMachine", debug=True)

with my_qbob.allocate_qubits("q", 3) as q:
    my_qbob += H(q[0])
    my_qbob.log_state(q[0])
    my_qbob += H(q[1])
    my_qbob.log_state(q[1])
    my_qbob += X(q[2])
    my_qbob.log_state(q[2])
    my_qbob.returns([M(q[0]), M(q[1]), M(q[2])])

In [9]:
my_qpam = qpam.ProgramArchitect("TestDumpMachine")
my_qpam.add_operations(my_qbob)
project_file = my_qpam.create_project(".")

In [10]:
!cat $project_file

<Project Sdk="Microsoft.Quantum.Sdk/0.14.2011120240">
    <PropertyGroup>
        <OutputType>Exe</OutputType>
        <TargetFramework>netcoreapp3.1</TargetFramework>
    </PropertyGroup>
</Project>

In [11]:
!cat Program.qs

namespace TestDumpMachine {
    open Microsoft.Quantum.Canon;
    open Microsoft.Quantum.Intrinsic;

    operation TestDumpMachine () : Result[] {
        using (q = Qubit[3]) {
            H(q[0]);
            // # wave function for qubits with ids (least to most significant): 0
            // ∣0❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
            // ∣1❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
            H(q[1]);
            // # wave function for qubits with ids (least to most significant): 1
            // ∣0❭:	 0.707107 +  0.000000 i	 == 	**********           [ 0.500000 ]     --- [  0.00000 rad ]
            // ∣1❭:	 0.707107 +  0.000000 i	 == 	**********           [ 0.500000 ]     --- [  0.00000 rad ]
            X(q[2]);
            // # wave function for qubits with ids (least to most significant): 2
            // ∣0❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 